In [10]:
import tensorflow as tf

def setup_gpu():
    physical_devices = tf.config.list_physical_devices('GPU')
    if physical_devices:
        try:
            tf.config.experimental.set_memory_growth(physical_devices[0], True)
            print("config augmentation allocation mémoire gpu activée")
        except RuntimeError as e:
            print(e)

setup_gpu()

In [11]:
import keras
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sea
import sklearn
import scipy as sc
import nltk as nltk
import statsmodels as statsmodels
import os
import warnings

pd.set_option('display.max_rows', 50)
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [12]:
print("Version de TensorFlow :", tf.__version__)

Version de TensorFlow : 2.10.1


In [13]:
print("Version de Keras :", keras.__version__)

Version de Keras : 2.10.0


In [14]:
projectPath = "/home/carolus/Documents/school/green_ia/"
fileNbr = '00' # numéro d'identification des csv à charger et générer
csvPath = projectPath + "data/" + fileNbr + "_openfoodfacts_00/" 

colToSave = ['allergens_from_ingredients',
            'nutriscore_tags',
            'labels_old',
            'categories_old',
            'pnns_groups_1',
            'ecoscore_data',
            'brand_owner_imported',
            'ingredients_tags',
            'packaging',
            'ingredients_hierarchy',
            'product_name',
            'food_groups_tags',
            'ecoscore_tags',
            'nova_group',
            'ingredients_from_or_that_may_be_from_palm_oil_n',
            'categories_tags',
            'brand_owner',
            'nutrient_levels_tags',
            'allergens_tags',
            'ecoscore_extended_data',
            'categories',
            'nutriments',
            'nutriscore_2021_tags',
            'additives_old_n',
            'ecoscore_score',
            'labels_tags',
            'countries']

In [15]:
# compter les fichiers csv dans le dossier traité 
def countCsv(directory):
    csvNbr = 0
    for csvFile in os.listdir(directory):
        if csvFile.endswith('.csv'):
            csvNbr += 1
    return csvNbr

In [16]:
# lister noms de colonnes et les sauvegarder dans un fichier texte
def findAndSaveCol(df, csvPath, currentCsv):
    colName = df.columns.tolist()
    print(f"colName: {colName}")

    colTextFile = csvPath + "colSaver.txt"
    with open(colTextFile, 'w') as file:
        for name in colName:
            file.write(name + " | ")

In [17]:
def deleteFile(filePath):
    if os.path.exists(filePath):
        os.remove(filePath)
        print(f"fichier {filePath} supprimé")
    else:
        print(f"erreur, fichier {filePath} n'existe pas")

In [18]:
# compte le nombre de csv dans dossier
csvNbr = countCsv(csvPath)
print(f'nombre csv répertoire: {csvNbr}, fileNbr: {fileNbr}')

csvIterator = 1
while csvIterator < csvNbr:
    # initialise df
    df_00, df_01 = pd.DataFrame(), pd.DataFrame()

    currentCsv = f"{csvPath}{csvIterator}_openfoodfacts_00.csv"
    print(f"current csv: {currentCsv}")

    # traitement ici 
    df_00 = pd.read_csv(currentCsv)

    # lister toutes les colonnes du premier csv et les sauvegarder dans un fichier texte
    if csvIterator == 1:
        findAndSaveCol(df_00, csvPath, currentCsv)

    # garde dans df uniquement les colonnes utiles
    df_01 = df_00[colToSave]

    # générer un fichier csv bis avec les colonnes utiles uniquement
    df_01.to_csv(f"{csvPath}{csvIterator}_openfoodfacts_01.csv", index=False)

    # supprimer le fichier csv initial 
    deleteFile(currentCsv)

    csvIterator+=1

nombre csv répertoire: 662, fileNbr: 00
current csv: /home/carolus/Documents/school/green_ia/data/00_openfoodfacts_00/1_openfoodfacts_00.csv
colName: ['other_nutritional_substances_tags', 'ingredients_text_en_ocr_1563429232', 'correctors_tags', 'ingredients_text_en_ocr_1582131773_result', 'origin_it', 'ingredients_text_fr_ocr_1586424882', 'allergens_from_ingredients', 'ingredients_text_en_ocr_1562469025_result', 'ingredients_text_en_ocr_1550522058', 'informers_tags', 'ingredients_text_en_ocr_1550522122_result', 'ingredients_text_fr_ocr_1544477421_result', 'ingredients_text_en_ocr_1550522065_result', 'nutriscore_tags', 'ingredients_text_fr_ocr_1550256782', 'labels_old', 'ingredients_text_en_ocr_1585414184', 'product_name_xx_debug_tags', 'product_name_fr', 'allergens', 'categories_old', 'sources_fields', 'environment_impact_level_tags', 'stores', 'pnns_groups_1', 'ingredients_text_en_ocr_1584810972', 'popularity_tags', 'ingredients_text_en_ocr_1619260168', 'product_name_nl_debug_tags', '

FileNotFoundError: [Errno 2] No such file or directory: '/home/carolus/Documents/school/green_ia/data/00_openfoodfacts_00/332_openfoodfacts_00.csv'

In [ ]:
# afficher contenu d'une colonne
#print(df['countries'].sample(n=18, random_state=50))

In [ ]:
#df.tail(10)

In [ ]:
#tempTest = f"{csvPath}1_openfoodfacts_01.csv"
#print(f"current csv: {tempTest}")
#dfTest = pd.read_csv(tempTest)
#dfTest.tail(5)